In [1]:
import time

import jax
jax.config.update("jax_enable_x64", True)

import jax.numpy as jnp
import numpy as np

from vehicle_model_jax import evalf, compute_jacobian_jax, get_default_params

def is_close(y, yhat, eps=1e-10):
    return np.abs(y - yhat) < eps


p = get_default_params()
p_tuple = tuple(p.values())

x0 = jnp.zeros(10)
x0 = x0.at[0].set(1e-5)  # v

u = jnp.array([0.0, 0.0])

f = evalf(x0, p_tuple, u)

J_jax = compute_jacobian_jax(x0, p_tuple, u)

In [2]:
J_analytic = {
    (0, 0): lambda p, x: (-p['R_s'] - p['K_p_i']) / p['L_ds'],
    (0, 1): lambda p, x: (p['G'] / p['r_w'] * x[7]) * p['L_qs'] / p['L_ds'],
    (0, 2): lambda p, x: p['K_i_i'] / p['L_ds'],
    
    (1, 0): lambda p, x: -(p['G'] / p['r_w'] * x[7]) * p['L_ds'] / p['L_qs'],
    (1, 1): lambda p, x: (-p['R_s'] - p['K_p_i']) / p['L_qs'],
    (1, 3): lambda p, x: p['K_i_i'] / p['L_qs'],
    
    (2, 0): lambda p, x: -1.0,
    (3, 1): lambda p, x: -1.0,
    
    (4, 6): lambda p, x: -x[7] * np.sin(x[6]),
    (4, 7): lambda p, x: np.cos(x[6]),
    
    (5, 6): lambda p, x: x[7] * np.cos(x[6]),
    (5, 7): lambda p, x: np.sin(x[6]),
    
    (6, 8): lambda p, x: 1.0,
    
    (7, 1): lambda p, x: (p['k'] * (3/2) * p['p_pairs'] * p['lambda_f']) / p['m'],
    
    (8, 6): lambda p, x: -p['K_p_theta'] / p['I'],
    (8, 8): lambda p, x: (-p['K_d_theta'] - p['c_r']) / p['I'],
    
    (9, 7): lambda p, x: -1.0,
}

In [3]:
# 2. Constant Speed
n_trials = 1000
for _ in range(n_trials):
    v0 = np.random.rand() * 10.0
    
    x0 = jnp.zeros(10)
    x0 = x0.at[7].set(v0)  # v
    u = jnp.array([v0, 0.0])
    
    J_jax = compute_jacobian_jax(x0, p_tuple, u)
    for (i, j), J_val_func in J_analytic.items():
        # print(i,j)
        assert is_close(J_jax[i, j], J_val_func(p, x0), eps=1e-10)

print(f"Passed the constant-speed state test ({n_trials} trials)")


Passed the constant-speed state test (1000 trials)


In [4]:
p

{'m': 1500,
 'I': 2500,
 'c_d': 25,
 'c_r': 500,
 'R_s': 0.1,
 'L_ds': 0.002,
 'L_qs': 0.003,
 'lambda_f': 0.5,
 'p_pairs': 4,
 'G': 10.0,
 'r_w': 0.3,
 'eta_g': 0.95,
 'k': 31.666666666666668,
 'K_p_i': 10.0,
 'K_i_i': 100.0,
 'K_p_v': 100.0,
 'K_i_v': 100.0,
 'K_p_theta': 1000.0,
 'K_d_theta': 1000.0,
 'v_w': 0.0,
 'psi': 0.0,
 'c_wx': 0.5,
 'c_wy': 0.5}

In [5]:
25/1500

0.016666666666666666

In [6]:
x0 = x0.at[7].set(v0)  # v
evalf(x0, p_tuple, u)[7]

Array(-0.1200437, dtype=float64)

In [7]:
x0 = x0.at[7].set(v0 + 1e-5)  # v

evalf(x0, p_tuple, u)[7]

Array(-0.12004387, dtype=float64)

In [8]:
-0.06334509 - (-0.0633449)

-1.900000000110147e-07

In [9]:
J_jax[i, j]

Array(-1., dtype=float64)

In [10]:
J_val_func(p, x0)

-1.0

In [11]:

# 3. Acceleration
n_trials = 1000
for _ in range(n_trials):
    v0 = np.random.rand() * 10.0
    
    x0 = jnp.zeros(10)
    u = jnp.array([v0, 0.0])
    
    J_jax = compute_jacobian_jax(x0, p_tuple, u)
    for (i, j), J_val_func in J_analytic.items():
        assert is_close(J_jax[i, j], J_val_func(p, x0), eps=1e-10)

print(f"Passed the accelration test ({n_trials} trials)")

# 4. Spin
n_trials = 1000
for _ in range(n_trials):
    w0 = np.random.rand() * np.pi/2
    
    x0 = jnp.zeros(10)
    u = jnp.array([0.0, w0])
    
    J_jax = compute_jacobian_jax(x0, p_tuple, u)
    for (i, j), J_val_func in J_analytic.items():
        assert is_close(J_jax[i, j], J_val_func(p, x0), eps=1e-10)

print(f"Passed the spinning test ({n_trials} trials)")

# 5. Level Turn
n_trials = 1000
for _ in range(n_trials):
    v0 = np.random.rand() * 10.0
    w0 = np.random.rand() * np.pi/2
    
    x0 = jnp.zeros(10)
    x0 = x0.at[7].set(v0)  # v
    u = jnp.array([v0, w0])
    
    J_jax = compute_jacobian_jax(x0, p_tuple, u)
    for (i, j), J_val_func in J_analytic.items():
        assert is_close(J_jax[i, j], J_val_func(p, x0), eps=1e-10)

print(f"Passed the level turn test ({n_trials} trials)")

Passed the accelration test (1000 trials)
Passed the spinning test (1000 trials)
Passed the level turn test (1000 trials)


In [12]:
J_jax[i, j], J_val_func(p, x0)

(Array(-1., dtype=float64), -1.0)

In [13]:
!python test_jacobian.py

/Users/youngjinpark/miniconda3/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Passed the steady-state test
Passed the constant-speed state test (1000 trials)
Passed the accelration test (1000 trials)
Passed the spinning test (1000 trials)
Passed the level turn test (1000 trials)
